In [10]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from IPython.display import display

from glob import glob


def parse_energy_file(file_path):
    metrics = {}
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                parts = line.split()
                if len(parts) < 2:
                    continue
                
                full_metric = parts[0]
                value = float(parts[-1])
                
                if "{" in full_metric and "}" in full_metric:
                    metric_name = full_metric.split("{")[0]
                    labels = full_metric.split("{")[1].split("}")[0]
                    
                    if "mode=" in labels:
                        mode = labels.split("mode=")[1].split(",")[0].strip('"')
                        key = f"{metric_name}_{mode}"
                    else:
                        key = metric_name
                else:
                    key = full_metric
                
                metrics[key] = value
            except (IndexError, ValueError) as e:
                print(f"Error parsing line '{line}': {str(e)}")
                continue
    return metrics


def calculate_energy_consumption(before_metrics, after_metrics):
    if not before_metrics or not after_metrics:
        return None

    total_joules_key = "kepler_container_joules_total_dynamic"

    dynamic_key = f'{total_joules_key}'
    if dynamic_key in after_metrics and dynamic_key in before_metrics:
        total_energy = after_metrics[dynamic_key] - before_metrics[dynamic_key]
    else:
        return None

    return total_energy if total_energy >= 0 else None
def calculate_cpu_usage_time(before_metrics, after_metrics):
    if not before_metrics or not after_metrics:
        return None

    cpu_usage_key = "kepler_container_bpf_cpu_time_ms_total"

    if cpu_usage_key in after_metrics and cpu_usage_key in before_metrics:
        total_cpu_time = after_metrics[cpu_usage_key] - before_metrics[cpu_usage_key]
    else:
        return None

    return total_cpu_time if total_cpu_time >= 0 else None

def process_test_directory(base_dir, test_name, requests, gateway=False):
    """Process all report directories for a specific test case."""
    energy_consumptions = []
    cpu_time_consumptions = []

    # Determine the path pattern based on whether it's gateway or not
    path_pattern = os.path.join(base_dir, str(requests), "report_*")

    # Process each report directory
    for report_dir in glob(path_pattern):
        try:
            before_file_gateway = None
            after_file_gateway = None
            before_metrics_gateway = None
            after_metrics_gateway = None
            energy_gateway = 0
            
            if gateway:
                before_file_gateway = os.path.join(report_dir, "before_gateway.txt")
                after_file_gateway = os.path.join(report_dir, "after_gateway.txt")
                
            before_file = os.path.join(report_dir, "before.txt")
            after_file = os.path.join(report_dir, "after.txt")

            if os.path.exists(before_file) and os.path.exists(after_file):
                if gateway and os.path.exists(before_file_gateway) and os.path.exists(after_file_gateway):
                    before_metrics_gateway = parse_energy_file(before_file_gateway)
                    after_metrics_gateway = parse_energy_file(after_file_gateway)
                    energy_gateway = calculate_energy_consumption(
                        before_metrics_gateway, after_metrics_gateway
                    )
                
                
                before_metrics = parse_energy_file(before_file)
                after_metrics = parse_energy_file(after_file)
                energy = calculate_energy_consumption(before_metrics, after_metrics)
                cpu_time = calculate_cpu_usage_time(before_metrics, after_metrics)
                if energy is not None:
                    energy_consumptions.append(energy + energy_gateway)
                if cpu_time is not None:
                    cpu_time_consumptions.append(cpu_time)
        except Exception as e:
            print(f"Error processing {report_dir}: {str(e)}")
            continue

    if not energy_consumptions:
        return None
    if not cpu_time_consumptions:
        return None
    
    # Pearson correlation coefficient
    if len(energy_consumptions) != len(cpu_time_consumptions):
        print(f"Warning: Mismatched lengths for energy and CPU time in {test_name} with {requests} requests.")
        return None
    res = stats.spearmanr(cpu_time_consumptions, energy_consumptions, alternative='greater')
    
    mean_value = np.mean(energy_consumptions)
    std_value = np.std(energy_consumptions)
    ste_value = std_value / np.sqrt(len(energy_consumptions))
    rsd = (std_value / mean_value) * 100 if mean_value != 0 else 0
    rse = (ste_value / mean_value) * 100 if mean_value != 0 else 0


    # Calculate statistics
    statistics = {
        "Test": test_name,
        "Requests": requests,
        "Mean": round(np.mean(energy_consumptions), 2),
        "Med": round(np.median(energy_consumptions), 2),
        "Min": round(np.min(energy_consumptions), 2),
        "Max": round(np.max(energy_consumptions), 2),
        "Std": round(np.std(energy_consumptions), 2),
        "RSD (%)": round(rsd, 2),
        "RSE (%)": round(rse, 2),
        "CPU Time (ms)": round(np.mean(cpu_time_consumptions), 2),
        "Monotonic relationship p-value":  round(res.pvalue, 6),
        "Monotonic relationship coefficient": round(res.correlation, 4),
        
        
    }

    return statistics


def get_endpoint_name(path):
    """Extract endpoint name from path"""
    parts = path.split("/")
    # Look for parts like 'json_get_user_id' or 'protobuf_get_all_users'
    for part in parts:
        if part.startswith("json_") or part.startswith("protobuf_"):
            return part.replace("json_", "").replace("protobuf_", "")
    return "unknown_endpoint"


def custom_sort(row):
    """Custom sorting function for the desired order"""
    test = row["Test"]
    requests = row["Requests"]

    if "Gateway" not in test:
        if requests == 1000:
            return 0 if "JSON" in test else 1
        else:
            return 2 if "JSON" in test else 3
    else:
        if requests == 1000:
            return 4 if "JSON" in test else 5
        else:
            return 6 if "JSON" in test else 7


test_dirs = [
    "../Control_Project_JSON/k6/json_get_all_products_entities/reports/json_get_all_products_entities",
    "../Control_Project_JSON/k6/json_get_user_id/reports/json_get_user_id",
    "../Control_Project_JSON/k6/json_get_all_users/reports/json_get_all_users",
    "../Control_Project_JSON/k6/json_create_user/reports/json_create_user",
    "../Control_Project_JSON/k6/json_update_user/reports/json_update_user",
    "../Control_Project_JSON/k6/json_delete_user/reports/json_delete_user",
    "../Experimental_Group_ProtoBuff/k6/protobuf_get_all_products_entities/reports/protobuf_get_all_products_entities",
    "../Experimental_Group_ProtoBuff/k6/protobuf_get_all_users/reports/protobuf_get_all_users",
    "../Experimental_Group_ProtoBuff/k6/protobuf_get_user_id/reports/protobuf_get_user_id",
    "../Experimental_Group_ProtoBuff/k6/protobuf_create_user/reports/protobuf_create_user",
    "../Experimental_Group_ProtoBuff/k6/protobuf_update_user/reports/protobuf_update_user",
    "../Experimental_Group_ProtoBuff/k6/protobuf_delete_user/reports/protobuf_delete_user",
    "../Gateway/k6_json/json_get_all_products_entities/reports/json_get_all_products_entities_gateway",
    "../Gateway/k6_json/json_get_all_users/reports/json_get_all_users_gateway",
    "../Gateway/k6_json/json_get_user_id/reports/json_get_user_id_gateway",
    "../Gateway/k6_json/json_create_user/reports/json_create_user_gateway",
    "../Gateway/k6_json/json_update_user/reports/json_update_user_gateway",
    "../Gateway/k6_json/json_delete_user/reports/json_delete_user_gateway",
    "../Gateway/k6_protobuf/protobuf_get_all_products_entities/reports/protobuf_get_all_products_entities_gateway",
    "../Gateway/k6_protobuf/protobuf_get_all_users/reports/protobuf_get_all_users_gateway",
    "../Gateway/k6_protobuf/protobuf_get_user_id/reports/protobuf_get_user_id_gateway",
    "../Gateway/k6_protobuf/protobuf_create_user/reports/protobuf_create_user_gateway",
    "../Gateway/k6_protobuf/protobuf_update_user/reports/protobuf_update_user_gateway",
    "../Gateway/k6_protobuf/protobuf_delete_user/reports/protobuf_delete_user_gateway",
]

# Group by endpoint
endpoint_data = {}

for test_dir in test_dirs:
    if not os.path.exists(test_dir):
        print(f"Directory not found: {test_dir}")
        continue

    # Determine endpoint name, test type, and whether it's gateway
    endpoint = get_endpoint_name(test_dir)
    is_gateway = "Gateway" in test_dir
    is_protobuf = "protobuf" in test_dir.lower()

    test_type = "PROTOBUF" if is_protobuf else "JSON"
    if is_gateway:
        test_type += " + Gateway"

    # Process for both request counts
    for requests in [1000, 10000]:
        statistics = process_test_directory(test_dir, test_type, requests, is_gateway)
        if statistics:
            if endpoint not in endpoint_data:
                endpoint_data[endpoint] = []
            endpoint_data[endpoint].append(statistics)

# Print results for each endpoint
for endpoint, results in endpoint_data.items():
    df = pd.DataFrame(results)

    # Add sorting key and sort
    df["sort_key"] = df.apply(custom_sort, axis=1)
    df = df.sort_values("sort_key").drop("sort_key", axis=1)

    df = df[
        [
            "Test",
            "Requests",
            "Mean",
            "Med",
            "Min",
            "Max",
            "Std",
            "RSD (%)",
            "RSE (%)",
            "CPU Time (ms)",
            "Monotonic relationship p-value",
            "Monotonic relationship coefficient",
        ]
    ]

    print(
        f"\nEnergy Consumption Statistics for {endpoint.replace('_', ' ').title()} (Joules):"
    )
    display(df)

    comparisons = []
    for (is_gateway, req_count), group in df.groupby([df["Test"].str.contains("Gateway"), "Requests"]):
        if len(group) == 2:  
            json_row = group[group["Test"].str.contains("JSON")].iloc[0]
            proto_row = group[group["Test"].str.contains("PROTOBUF")].iloc[0]

            test_name = "PROTOBUF + Gateway vs JSON + Gateway" if is_gateway else "PROTOBUF vs JSON"
            diff_pcts = {
                "Test": f"{test_name} ({req_count} requests)",
                "Mean %": (
                    (json_row["Mean"] - proto_row["Mean"]) / json_row["Mean"]
                )
                * 100,
                "Median %": ((json_row["Med"] - proto_row["Med"]) / json_row["Med"])
                * 100,
                "Min %": ((json_row["Min"] - proto_row["Min"]) / json_row["Min"])
                * 100,  
                "Max %": ((json_row["Max"] - proto_row["Max"]) / json_row["Max"])
                * 100,
                "Std %": ((json_row["Std"] - proto_row["Std"]) / json_row["Std"])
                * 100,
            }
            comparisons.append(diff_pcts)

    # Create comparison table
    comparison_df = pd.DataFrame(comparisons)
    comparison_df = comparison_df.round(2)  # Round to 2 decimal places

    print(f"\nPercentage Comparison for {endpoint.replace('_', ' ').title()}:")
    display(comparison_df)


Energy Consumption Statistics for Get All Products Entities (Joules):


,Test,Requests,Mean,Med,Min,Max,Std,RSD (%),RSE (%),CPU Time (ms),Monotonic relationship p-value,Monotonic relationship coefficient
0,JSON,1000,20.94,12.09,5.63,86.82,23.14,110.55,34.96,3580.6,0.000000,0.9879
2,PROTOBUF,1000,16.09,13.42,0.59,59.94,16.18,100.53,31.79,2612.8,0.000000,0.9879
1,JSON,10000,80.86,81.83,73.28,86.62,4.35,5.38,1.70,11012.6,0.000000,1.0000
3,PROTOBUF,10000,73.04,73.14,68.29,80.64,3.70,5.06,1.60,9916.8,0.000011,0.9515
4,JSON + Gateway,1000,49.54,35.87,10.29,180.34,47.21,95.30,30.14,3715.1,0.000000,0.9879
6,PROTOBUF + Gateway,1000,28.16,27.53,9.51,68.07,16.25,57.71,18.25,2982.8,0.000004,0.9636
5,JSON + Gateway,10000,213.69,212.34,199.01,232.54,8.28,3.88,1.23,13447.2,0.000000,0.9879
7,PROTOBUF + Gateway,10000,202.94,206.12,190.59,211.11,7.56,3.73,1.18,12353.8,0.000011,0.9515



Percentage Comparison for Get All Products Entities:


,Test,Mean %,Median %,Min %,Max %,Std %
0,PROTOBUF vs JSON (1000 requests),23.16,-11.00,89.52,30.96,30.08
1,PROTOBUF vs JSON (10000 requests),9.67,10.62,6.81,6.90,14.94
2,PROTOBUF + Gateway vs JSON + Gateway (1000 req...,43.16,23.25,7.58,62.25,65.58
3,PROTOBUF + Gateway vs JSON + Gateway (10000 re...,5.03,2.93,4.23,9.22,8.70



Energy Consumption Statistics for Get User Id (Joules):


...

,Test,Mean %,Median %,Min %,Max %,Std %
0,PROTOBUF vs JSON (1000 requests),-4.42,-7.66,-2.43,1.93,30.71
1,PROTOBUF vs JSON (10000 requests),0.07,-0.08,1.23,-1.84,-32.30
2,PROTOBUF + Gateway vs JSON + Gateway (1000 req...,4.49,2.60,2.57,11.20,47.59
3,PROTOBUF + Gateway vs JSON + Gateway (10000 re...,-0.67,-0.12,-1.74,-0.46,22.51



Energy Consumption Statistics for Delete User (Joules):


,Test,Requests,Mean,Med,Min,Max,Std,RSD (%),RSE (%),CPU Time (ms),Monotonic relationship p-value,Monotonic relationship coefficient
0,JSON,1000,66.90,65.74,60.21,78.32,5.99,8.96,2.83,11699.4,0.000004,0.9636
2,PROTOBUF,1000,65.66,66.96,58.07,71.84,4.38,6.67,2.11,11484.8,0.000000,0.9879
1,JSON,10000,365.48,347.75,90.69,681.85,187.26,51.24,16.20,60751.8,0.000000,1.0000
3,PROTOBUF,10000,371.05,360.46,100.35,680.74,180.78,48.72,15.41,62211.0,0.000000,1.0000
4,JSON + Gateway,1000,85.17,83.97,72.06,97.46,6.84,8.03,2.54,12971.5,0.002431,0.8061
6,PROTOBUF + Gateway,1000,87.73,84.93,75.09,113.56,9.95,11.35,3.59,12750.1,0.000001,0.9758
5,JSON + Gateway,10000,535.65,530.50,270.46,795.14,163.78,30.58,9.67,68184.8,0.000000,1.0000
7,PROTOBUF + Gateway,10000,540.75,531.40,259.59,819.92,168.03,31.07,9.83,68034.4,0.000000,1.0000



Percentage Comparison for Delete User:


,Test,Mean %,Median %,Min %,Max %,Std %
0,PROTOBUF vs JSON (1000 requests),1.85,-1.86,3.55,8.27,26.88
1,PROTOBUF vs JSON (10000 requests),-1.52,-3.65,-10.65,0.16,3.46
2,PROTOBUF + Gateway vs JSON + Gateway (1000 req...,-3.01,-1.14,-4.20,-16.52,-45.47
3,PROTOBUF + Gateway vs JSON + Gateway (10000 re...,-0.95,-0.17,4.02,-3.12,-2.59
